<a href="https://colab.research.google.com/github/lamyse1/deep-learning-/blob/main/Week6/DL_Week6_Ex2_chapter10_dl_for_timeseries_i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AAI612: Deep Learning & its Applications

*Notebook 6.3: Understanding recurrent neural networks*



**Read and parse the data of the Jena weather dataset**

In [2]:
import ssl
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/harmanani/AAI612/main/Week6/data/jena_climate_2009_2016.csv"

ssl._create_default_https_context = ssl._create_unverified_context
f = urlopen(url)
data = f.read()

lines = data.decode().split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    temperature[i] = values[1]
    raw_data[i, :] = values[:]

num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451
num_train_samples: 210225
num_val_samples: 105112
num_test_samples: 105114


**Compute the number of samples we'll use for each data split**

In [4]:
import keras.utils

sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

train_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

**NumPy implementation of a simple RNN**

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
timesteps = 100
input_features = 32
output_features = 64
inputs = np.random.random((timesteps, input_features))
state_t = np.zeros((output_features,))
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))
successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t = output_t
final_output_sequence = np.stack(successive_outputs, axis=0)

### A recurrent layer in Keras

**An RNN layer that can process sequences of any length**

In [6]:
num_features = 14
inputs = keras.Input(shape=(None, num_features))
outputs = layers.SimpleRNN(16)(inputs)

**An RNN layer that returns only its last output step**

In [7]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=False)(inputs)
print(outputs.shape)

(None, 16)


**An RNN layer that returns its full output sequence**

In [8]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs)
print(outputs.shape)

(None, 120, 16)


**Stacking RNN layers**

In [9]:
inputs = keras.Input(shape=(steps, num_features))
x = layers.SimpleRNN(16, return_sequences=True)(inputs)
x = layers.SimpleRNN(16, return_sequences=True)(x)
outputs = layers.SimpleRNN(16)(x)

## Advanced use of recurrent neural networks

### Using recurrent dropout to fight overfitting

**Training and evaluating a dropout-regularized LSTM**

In [ ]:
sequence_length = 120

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=callbacks)

Epoch 1/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 153s 183ms/step - loss: 105.3189 - mae: 8.2555 - val_loss: 41.7394 - val_mae: 5.0458
Epoch 2/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 201s 182ms/step - loss: 51.5413 - mae: 5.5818 - val_loss: 29.2365 - val_mae: 4.2302
Epoch 3/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 163s 199ms/step - loss: 42.5867 - mae: 5.0537 - val_loss: 21.7118 - val_mae: 3.5278
Epoch 4/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 148s 180ms/step - loss: 39.7507 - mae: 4.8855 - val_loss: 20.8236 - val_mae: 3.5274
Epoch 5/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 150s 183ms/step - loss: 38.8931 - mae: 4.8393 - val_loss: 22.4874 - val_mae: 3.7345
Epoch 6/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 149s 182ms/step - loss: 38.3420 - mae: 4.7932 - val_loss: 25.6884 - val_mae: 4.0652
Epoch 7/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 163s 199ms/step - loss: 38.2484 - mae: 4.7844 - val_loss: 19.4372 - val_mae: 3.4083
Epoch 8/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 148s 180ms/step - loss: 37.8002 - mae: 4.7572 - val_loss: 19.3838 - val_mae: 3.4220
Epoch 9

In [ ]:
inputs = keras.Input(shape=(sequence_length, num_features))
x = layers.LSTM(32, recurrent_dropout=0.2, unroll=True)(inputs)

### Stacking recurrent layers

**Training and evaluating a dropout-regularized, stacked GRU model**

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 251s 299ms/step - loss: 56.4791 - mae: 5.7452 - val_loss: 21.0452 - val_mae: 3.6194
Epoch 2/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 258s 294ms/step - loss: 23.7852 - mae: 3.8563 - val_loss: 17.7771 - val_mae: 3.3013
Epoch 3/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 246s 300ms/step - loss: 23.3388 - mae: 3.8169 - val_loss: 19.3402 - val_mae: 3.4555
Epoch 4/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 260s 297ms/step - loss: 22.9988 - mae: 3.7983 - val_loss: 17.6210 - val_mae: 3.2871
Epoch 5/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 258s 293ms/step - loss: 22.6119 - mae: 3.7525 - val_loss: 17.0765 - val_mae: 3.2105
Epoch 6/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 264s 295ms/step - loss: 22.4251 - mae: 3.7366 - val_loss: 18.0137 - val_mae: 3.3435
Epoch 7/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 243s 296ms/step - loss: 22.0421 - mae: 3.7103 - val_loss: 18.6777 - val_mae: 3.4039
Epoch 8/50
819/819 ━━━━━━━━━━━━━━━━━━━━ 242s 296ms/step - loss: 21.6022 - mae: 3.6762 - val_loss: 17.2323 - val_mae: 3.2215
Epoch 9/

### Using bidirectional RNNs

**Training and evaluating a bidirectional LSTM**

In [10]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.Bidirectional(layers.LSTM(16))(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset)

Epoch 1/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 142s 169ms/step - loss: 109.4151 - mae: 8.4838 - val_loss: 57.7342 - val_mae: 6.1306
Epoch 2/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 136s 166ms/step - loss: 59.4063 - mae: 6.0730 - val_loss: 42.4590 - val_mae: 5.1656
Epoch 3/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 136s 166ms/step - loss: 49.1452 - mae: 5.4131 - val_loss: 35.6415 - val_mae: 4.6782
Epoch 4/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 139s 169ms/step - loss: 41.4281 - mae: 4.9114 - val_loss: 29.8749 - val_mae: 4.2427
Epoch 5/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 138s 168ms/step - loss: 35.4405 - mae: 4.5192 - val_loss: 26.6965 - val_mae: 4.0179
Epoch 6/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 139s 170ms/step - loss: 30.9284 - mae: 4.2270 - val_loss: 24.1940 - val_mae: 3.7863
Epoch 7/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 138s 168ms/step - loss: 27.6501 - mae: 4.0117 - val_loss: 23.7829 - val_mae: 3.8072
Epoch 8/10
819/819 ━━━━━━━━━━━━━━━━━━━━ 142s 169ms/step - loss: 25.2994 - mae: 3.8613 - val_loss: 21.6911 - val_mae: 3.6349
Epoch 9